In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from keras.layers import Input

In [2]:
# Load the dataset
df = pd.read_csv('bitcoin_price_dataset.csv')

In [4]:
# Convert timestamp to datetime and set it as index
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df.set_index('timestamp', inplace=True)

In [5]:
# We'll use 'close' price for prediction
data = df['close'].values.reshape(-1, 1)

In [6]:
# Normalize the data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

In [7]:
    # Function to create sequences
    def create_sequences(data, seq_length):
        X = []
        y = []
        for i in range(len(data) - seq_length):
            X.append(data[i:(i + seq_length), 0])
            y.append(data[i + seq_length, 0])
        return np.array(X), np.array(y)

In [8]:
# Create sequences
seq_length = 60  # Use last 60 days to predict the next day
X, y = create_sequences(data_normalized, seq_length)

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [13]:
# Build the LSTM model
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    LSTM(units=50, return_sequences=True),
    LSTM(units=50),
    Dense(1)
])

In [14]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

In [ ]:
# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

In [ ]:
# Inverse transform predictions
train_predictions = scaler.inverse_transform(train_predictions)
y_train = scaler.inverse_transform([y_train])
test_predictions = scaler.inverse_transform(test_predictions)
y_test = scaler.inverse_transform([y_test])

In [ ]:
# Calculate RMSE
train_rmse = np.sqrt(np.mean((train_predictions - y_train.T)**2))
test_rmse = np.sqrt(np.mean((test_predictions - y_test.T)**2))
print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

In [ ]:
# Plot the results
plt.figure(figsize=(16,8))
plt.plot(df.index[-len(y_test):], y_test.T, label='True Price')
plt.plot(df.index[-len(y_test):], test_predictions, label='Predicted Price')
plt.title('Bitcoin Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Plot training history
plt.figure(figsize=(16,8))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()